In [1]:
!pwd

/home/iceberg/notebooks


In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("spark-iceberg-minio-dev")

    # Iceberg catalog (HadoopCatalog on MinIO)
    .config("spark.sql.catalog.local_minio", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local_minio.catalog-impl", "org.apache.iceberg.hadoop.HadoopCatalog")
    .config("spark.sql.catalog.local_minio.warehouse", "s3a://warehouse/")

    # Iceberg SQL extensions (mandatory)
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )

    .getOrCreate()
)

[2025-12-29 07:15:13] [Thread-3] WARN  org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[2025-12-29 07:15:13] [Thread-3] WARN  org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
hadoop_conf = spark._jsc.hadoopConfiguration()

hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.access.key", "minioadmin")
hadoop_conf.set("fs.s3a.secret.key", "minioadmin")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Good defaults
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")
hadoop_conf.set("fs.s3a.aws.credentials.provider",
                 "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

In [4]:
spark.sparkContext.setLogLevel("WARN")

In [5]:
spark.version

'4.0.1'

In [6]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [7]:
spark.read.text("s3a://warehouse/").show(5)

[2025-12-29 07:15:22] [Thread-3] WARN  org.apache.hadoop.metrics2.impl.MetricsConfig - Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
[2025-12-29 07:15:22] [Thread-3] WARN  org.apache.hadoop.metrics2.impl.MetricsConfig - Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+-----+
|value|
+-----+
+-----+

